In [6]:
import tkinter as tk
from tkinter import filedialog, Label, Button
from PIL import Image, ImageTk
import tensorflow as tf
import numpy as np
from tensorflow.keras import backend as K
import cv2

In [7]:
@tf.keras.utils.register_keras_serializable()
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        focal_factor = K.pow(1 - y_pred, gamma)
        loss = alpha * focal_factor * cross_entropy
        return K.sum(loss, axis=1)
    return focal_loss_fixed

In [8]:
class_labels = {
    0: "AD",  # Alzheimer's Disease
    1: "MCI", # Mild Cognitive Impairment
    2: "NC"   # Normal Cognition
}

In [9]:
#Filters
def median_Gaussian_filters(images):
    denoised_images = []
    for img in images:
        img = cv2.medianBlur(img.astype(np.uint8), 3)
        img = cv2.GaussianBlur(img, (5, 5), 10)
        denoised_images.append(img)
    return np.array(denoised_images)

def unsharp_mask(image, sigma=1.0, strength=1):
    if not isinstance(image, np.ndarray):
        image = np.array(image)
    kernel_size = int(6 * sigma + 1)
    if kernel_size % 2 == 0:
        kernel_size += 1
    blurred = cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)
    sharpened = cv2.addWeighted(image, 1.0 + strength, blurred, -strength, 0)
    return sharpened

In [10]:
MODEL_PATH = "C://Users//محمد عبدالفتاح//Desktop//Alzheimers-disease//3- Final//CNN code & Model//cnnModel.keras"
model = tf.keras.models.load_model(MODEL_PATH, custom_objects={'focal_loss': focal_loss, 'focal_loss_fixed': focal_loss()})

root = tk.Tk()
root.title("Image Classifier")
root.geometry("500x700")
root.configure(bg="#f0f0f5")

title_label = Label(root, text="Alzheimer's Image Classifier", font=("Helvetica", 20, "bold"), bg="#f0f0f5", fg="#2b2b52")
title_label.pack(pady=20)

label = Label(root, text="Upload an image to classify", font=("Helvetica", 14), bg="#f0f0f5", fg="#4d4d4d")
label.pack(pady=10)

image_path = None

def classify_image():
    global image_path
    if image_path:
        img = Image.open(image_path).resize((256, 256))
        img = img.convert('L')
        img_array = np.array(img)
        #Filters
        img_array = median_Gaussian_filters(img_array)
        img_array = unsharp_mask(img_array)
        img_array = img_array/ 255.0
        img_array = np.expand_dims(img_array, axis=0)
        predictions = model.predict(img_array)
        class_idx = np.argmax(predictions[0])
        confidence = predictions[0][class_idx]
        class_name = class_labels.get(class_idx, "Unknown Class")
        result_label.config(text=f"Class: {class_name}\nConfidence: {confidence:.2f}", fg="#2b9348")
    else:
        result_label.config(text="Please upload an image first!", fg="#c1121f")

def upload_image():
    global image_path
    image_path = filedialog.askopenfilename(
        filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")]
    )
    if image_path:
        img = Image.open(image_path)
        img.thumbnail((300, 300))
        img_tk = ImageTk.PhotoImage(img)
        image_label.config(image=img_tk)
        image_label.image = img_tk
        result_label.config(text="", fg="#4d4d4d")

upload_button = Button(
    root, text="Upload Image", command=upload_image,
    font=("Helvetica", 12, "bold"), bg="#5f6caf", fg="white", activebackground="#7a86b6", activeforeground="white"
)
upload_button.pack(pady=20)

image_label = Label(root, bg="#f0f0f5")
image_label.pack(pady=10)

classify_button = Button(
    root, text="Classify Image", command=classify_image,
    font=("Helvetica", 12, "bold"), bg="#5f6caf", fg="white", activebackground="#7a86b6", activeforeground="white"
)
classify_button.pack(pady=20)

result_label = Label(root, text="", font=("Helvetica", 14), bg="#f0f0f5", fg="#4d4d4d")
result_label.pack(pady=20)

root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
